# model save / load

### 생성한 모델을 disk 에 저장하고 복원

- model 의 architecture 저장 : json, yaml format  

- model 의 weigth 저장 : h5 file format  

- hdf5(Hierarchical Data Format version 5)는 대용량 데이터를 저장하기 위한 파일 포맷

In [1]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.layers import Conv2D, AveragePooling2D
from tensorflow.keras.layers import Dense, Flatten, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.datasets import mnist
from tensorflow.keras import utils

(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.astype("float32")
X_test = X_test.astype("float32")
X_train /= 255
X_test /= 255

X_train = X_train[:, :, :, np.newaxis]
X_test = X_test[:, :, :, np.newaxis]
y_train = utils.to_categorical(y_train)
y_test = utils.to_categorical(y_test)

# LeNet
model = tf.keras.Sequential()

model.add(Conv2D(6, kernel_size=5, padding="same", input_shape=(28, 28, 1)))
model.add(Activation("relu"))

model.add(AveragePooling2D(pool_size=(2, 2), strides=(1, 1), padding="valid"))

model.add(Conv2D(16, kernel_size=5, padding="valid"))
model.add(Activation('relu'))

model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding="valid"))

model.add(Flatten())

model.add(Dense(120))
model.add(Activation("relu"))

model.add(Dense(84))
model.add(Activation("relu"))

model.add(Dense(10))
model.add(Activation("softmax"))

model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train, y_train, batch_size=128, epochs=1, validation_data=(X_test, y_test))

score = model.evaluate(X_test, y_test, verbose=0)
print(score[0])
print(score[1])

Train on 60000 samples, validate on 10000 samples
60000/60000 [==============================] - 8s 141us/sample - loss: 0.2492 - accuracy: 0.9269 - val_loss: 0.0781 - val_accuracy: 0.9748
0.07808350970651955
0.9748


### Model Architecture Save

In [2]:
json_string = model.to_json()

In [3]:
with open("lenet.json", "w") as json_file:
    json_file.write(json_string)

### weight 만 save

In [4]:
model.save_weights("lenet_weight.h5")

### 전체 model save

In [5]:
model.save("lenet_model.h5")

## Model Load

- architecture load + weight load + compile

In [6]:
from tensorflow.keras.models import model_from_json

json_file = open("lenet.json", "r")
read_model = json_file.read()
json_file.close()

loaded_model = model_from_json(read_model)
loaded_model.load_weights("lenet_weight.h5")

loaded_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])

In [7]:
score = loaded_model.evaluate(X_test, y_test, verbose=0)

In [8]:
from tensorflow.keras.models import load_model

loaded_model = load_model('lenet_model.h5')

In [9]:
score = loaded_model.evaluate(X_test, y_test, verbose=0)

In [10]:
y_predict = model.predict(X_test)
y_predict

array([[3.2382490e-05, 4.0795661e-07, 1.7605283e-03, ..., 9.9779922e-01,
        4.8860955e-05, 9.2082912e-05],
       [2.1599875e-05, 6.3068280e-04, 9.9928468e-01, ..., 2.5984212e-08,
        5.8118676e-05, 7.1747641e-10],
       [4.9035916e-05, 9.9803251e-01, 1.8827243e-04, ..., 7.5606100e-04,
        2.3762397e-04, 2.5275156e-05],
       ...,
       [1.8391345e-08, 5.7595857e-09, 9.3870751e-09, ..., 7.4103418e-06,
        2.4578021e-06, 4.8049045e-04],
       [1.1206575e-06, 6.1447949e-08, 1.7411312e-08, ..., 3.9180982e-08,
        3.9386345e-04, 4.1280234e-07],
       [5.2431919e-06, 4.1509761e-11, 5.3404747e-06, ..., 1.5383439e-10,
        1.2697888e-07, 1.7567718e-08]], dtype=float32)

In [11]:
y_test

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [12]:
np.argmax(y_predict, axis=1)

array([7, 2, 1, ..., 4, 5, 6], dtype=int64)

In [13]:
np.argmax(y_test, axis=1)

array([7, 2, 1, ..., 4, 5, 6], dtype=int64)

In [14]:
np.sum(np.argmax(y_predict, axis=1) == np.argmax(y_test, axis=1)) / len(y_test) * 100

97.48

In [15]:
loaded_model.predict_classes(X_test[:1,:])

array([7], dtype=int64)